# 10.4 Deep Field Simulations of the WFI

In the next step of the tutorial we will simulate and analyse deep field observations with the Athena WFI. We
separate this into the generation of the simulation input (Sect. 10.4.1), which is fully independent of the chosen
instrument, followed by an observation with all four chips of the WFI (Sect. 10.4.2). Of course, this second step
can also be done with any other satellite and detector by simply replacing the chosen XML files.

In [ ]:
import copy
import glob
import matplotlib.colors as colors
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib import rc
import numpy as np
import os
import pandas
import time
import tempfile
import shutil, shlex
from math import radians


from astropy.io import fits
from astropy.visualization import astropy_mpl_style
from datetime import datetime
from funcs import *
from subprocess import check_call, STDOUT
from xspec import Xset, Plot, AllData, ModelManager, Spectrum, Model, AllModels, Fit

plt.style.use(astropy_mpl_style)



In [ ]:
tmpDir = tempfile.mkdtemp()
os.environ["PFILES"] = tmpDir + ":" + os.environ["PFILES"]
os.environ["HEADASNOQUERY"] = ""
os.environ["HEADASPROMPT"] = "/dev/null/"
SIXTE = os.environ["SIXTE"]
xmldir = f"{SIXTE}/share/sixte/instruments/athena-wfi/wfi_wo_filter_B4C"

xml0=f"{xmldir}/ld_wfi_ff_chip0.xml"
xml1=f"{xmldir}/ld_wfi_ff_chip1.xml"
xml2=f"{xmldir}/ld_wfi_ff_chip2.xml"
xml3=f"{xmldir}/ld_wfi_ff_chip3.xml"


## 10.4.1 SIMPUT for a Large Field

### Creating a SIMPUT for several sources with simputfile

(We'll use the spectral file *mcrab.xcm* that we created in previous tutorials)

In [ ]:
simpar="XSPECFile=mcrab.xcm Emin=0.5 Emax=10.0 clobber=yes"
ra = [40.21, 40.31, 40.12, 40.27, 40.29, 40.33]
dec = [12.82, 12.83, 12.73, 12.81, 12.73, 12.81]
src_flux = [8.3e-12, 2.3e-11, 6.3e-12, 4.1e-12, 3.2e-11, 1.3e-11]
l = 0
for i, j, k in zip(ra, dec, src_flux):
    sim_file = f"src_0{l}.fits"
    comm = (f"simputfile RA={i} Dec={j} srcFlux={k} Simput={sim_file} {simpar}")
    #print(comm)
    run_comm(comm, "Creating simput file for several sources")
    l = l + 1


In [ ]:
# Merge the simput files

opt="clobber=yes FetchExtensions=yes"

infile1 = "src_00.fits"
infile2 = "src_01.fits"
nfiles = len(ra) # from 00 to 05
for i in range(nfiles-1):
    if i == nfiles-2:
        outfile = "merged_simput.fits"
    else:
        outfile = f"m_src_0{i+1}.fits"
    comm = (f"simputmerge {opt} Infile1={infile1} Infile2={infile2} Outfile={outfile}")
    #print(comm)
    run_comm(comm,f'Merginig {infile1} and {infile2} into {outfile}')
    infile2 = outfile
    infile1 = f"src_0{i+2}.fits"


## 10.4.2 Simple Wide Field Simulations

### Viewing the chip geometry with `xml2svg`

In [ ]:
svg_file = "athenawfi_fullframe.svg"
pdf_file = "athenawfi_fullframe.pdf"

comm = (f"xml2svg XMLFiles='{xml0};{xml1};{xml2};{xml3}' SVGWidth=3000 Border=10 SVGName={svg_file}")
#print(comm)
run_comm(comm, 'Getting chip geometry')

# conver svg file to pdf
run_comm(f"convert {svg_file} {pdf_file}", 'Converting svg to pdf')
run_comm(f"xdg-open {pdf_file}", 'Open the pdf image in the default app')

### Simulating the four WFI chips with `athenwfisim`

In [ ]:
# Running simulation chip by chip with athenafism tool 
RApoint = 40.21
DECpoint = 12.77
comm = (f"athenawfisim XMLFile0={xml0} XMLFile1={xml1} XMLFile2={xml2} XMLFile3={xml3} RA={RApoint} Dec={DECpoint} Prefix='sim_' " +
        f"Simput=merged_simput.fits EvtFile=evt.fits Exposure=1000 clobber=yes ")
#print(comm)
run_comm(comm, 'Simulating 4 chips')

In [ ]:
#  merging the 4 chips
combined_file = 'sim_combined_evt.fits'
comm = (f'ftmerge sim_chip0_evt.fits,sim_chip1_evt.fits,sim_chip2_evt.fits,sim_chip3_evt.fits {combined_file} clobber=yes')
run_comm(comm, 'Merging 4 chips')

# To inspect the final evt file (same effect as fv/fstrct/fdump command)
f = fits.open(combined_file)
f.info()
print(f[1].columns.names)

### Creating an image with `imgev`

In [ ]:
# Creating image of the full WFI detector with imgev
img_file = 'sim_combined_img.fits'
comm = (f"imgev EvtFile={combined_file} Image={img_file} CoordinateSystem=0 Projection='TAN' NAXIS1=1063 NAXIS2=1063 " +
        f"CUNIT1='deg' CUNIT2='deg' CRVAL1={RApoint} CRVAL2={DECpoint} CRPIX1=532 CRPIX2=532 " + 
        f"CDELT1=-6.207043e-4 CDELT2=6.207043e-4 history=true clobber=yes")
#print(comm)
run_comm(comm, "Creating image of the WFIdetector")

In [ ]:
image_data = fits.getdata(img_file, ext=0)
fig, _axs = plt.subplots(nrows=1, ncols=2, figsize=(16*2,6*2))
fig.suptitle("Figure 17 (left): image of 4 WFI chip")
axs = _axs.flatten()
cmap = plt.cm.winter

cdelt = 6.207043e-4
crpix = 532

# Add secondary axes with coordinates (optional)
ra1 = RApoint + crpix*cdelt/np.cos(radians(DECpoint))
ra2 = RApoint - crpix*cdelt/np.cos(radians(DECpoint))
dec1 = DECpoint - crpix*cdelt
dec2 = DECpoint + crpix*cdelt
sq_downleft=np.array((ra1, dec1))
sq_upright=np.array((ra2, dec2))

draw_second_axis(fig, axs, sq_downleft=sq_downleft, sq_upright=sq_upright, ln=1063, mov_ra=20, mov_dec=5, 
                 lon_lines=17, num_lines=10, mov_x_text=[0, 67], mov_y_text=[17, 20], 
                 xlabel=r'$\alpha$/deg', ylabel=r'$\delta$/deg', num_axis=0, dist_txt=23, 
                 zero_between=False, display_format='.2f')
im = axs[0].imshow(image_data, cmap=cmap, norm=colors.LogNorm(vmin=1e-5, vmax=np.max(image_data), clip=True), origin="lower")
axs[0].set_ylim(0, len(image_data))
axs[0].set_xlim(0, len(image_data))
fig.colorbar(im,ax=axs[0], pad=0.1)

# zoomed image
x1 = crpix - 250; x2 = crpix+250; y1 = x1; y2 = x2
sq11 = RApoint + (crpix-x1)*cdelt/np.cos(radians(DECpoint))
sq12 = RApoint - (x2-crpix)*cdelt/np.cos(radians(DECpoint))
sq21 = DECpoint - (crpix-y1)*cdelt
sq22 = DECpoint + (y2-crpix)*cdelt
sq1 = np.array([sq11, sq21])
sq2 = np.array([sq12, sq22])

rect = patches.Rectangle(xy=(x1, y1), width=(x2-x1), height=(y2-y1), 
                         edgecolor='C0', facecolor='none', linewidth=2)
axs[0].add_patch(rect)

img2 = image_data[x1:x2, y1:y2]
axs[1].set_ylim(0, len(img2))
axs[1].set_xlim(0, len(img2))
im = axs[1].imshow(img2, cmap=cmap, norm=colors.LogNorm(vmin=1e-5, vmax=np.max(image_data), clip=True), origin="lower")
axs[1].set_yticklabels([])
axs[1].set_xticklabels([])
draw_second_axis(fig, axs, sq_downleft=sq1, sq_upright=sq2, ln=500, mov_ra=5, mov_dec=1, 
                 lon_lines=6, num_lines=10, mov_x_text=[0, 30], mov_y_text=[15, 13], 
                 xlabel=r'$\alpha$/deg', ylabel=r'$\delta$/deg', num_axis=1, dist_txt=10, 
                 zero_between=False, display_format='.3f')
fig.colorbar(im,ax=axs[1], pad=0.1)


### Creating a spectrum with `makespec`

In [ ]:
# Create spectrum for lower source at right side
combined_file = 'sim_combined_evt.fits'
spec_file = 'sim_combined_spec.pha'
comm = (f"makespec EvtFile={combined_file} Spectrum={spec_file} clobber=yes RSPPath={xmldir} EventFilter='(RA>40.10 && RA<40.14) && Dec>12.71 && Dec<12.75'")
#print(comm)
run_comm(comm, 'Generating spectrum')

In [ ]:
# shorten ARF and RMF paths
f = fits.open(spec_file)
rmf = f[1].header['RESPFILE']
arf = f[1].header['ANCRFILE']
f.close()

## create symlinks if not already done (uncomment)
#run_comm('ln -s {}'.format(rmf), 'Linking to working directory')
#run_comm('ln -s {}'.format(arf), 'Linking to working directory')

# strip path in keywords to avoid very long values and update header
rmf = os.path.basename(rmf)
arf = os.path.basename(arf)
f = fits.open(spec_file, 'update')
f[1].header['RESPFILE'] = rmf
f[1].header['ANCRFILE'] = arf
f.close()

In [ ]:
# rebin spectral data
binspec = "sim_combined_spec_rebin.pha"
comm = (f"grppha infile={spec_file} outfile={binspec} clobber=yes comm='group min 20 & exit'")
run_comm(comm, "Rebinning spectrum")

In [ ]:
# Rebin and Plot the spectrum
Plot.device = '/null'
AllData.clear()
AllModels.clear()
Plot.device = "/xs"
Plot.xAxis="keV"
s1 = Spectrum(binspec)
AllData.ignore("bad")
Plot("ldata")

## 10.4.3 Simulating the Chandra Deep Field South

Download Files: http://www.sternwarte.uni-erlangen.de/research/sixte/downloads/sixte/simput/CDFS_combined_simput.tgz


In [ ]:
# Running simulation chip by chip with athenafism tool 
RApoint = 53.13
DECpoint = -27.8
prefix = "cdfs_"
comm = (f"athenawfisim XMLFile0={xml0} XMLFile1={xml1} XMLFile2={xml2} XMLFile3={xml3} RA={RApoint} Dec={DECpoint} " +
        f"Prefix={prefix} Simput=CDFS_cat_lehmer.fits Simput2=CDFS_cat_galaxies.fits EvtFile=evt.fits Exposure=5000 clobber=yes")
#print(comm)
run_comm(comm, 'Simulating 4 chips')  

In [ ]:
# merging the event files
combined_file = f'{prefix}combined_evt.fits'
comm = (f"ftmerge {prefix}chip0_evt.fits,{prefix}chip1_evt.fits,{prefix}chip2_evt.fits,{prefix}chip3_evt.fits " +
        f"{combined_file} clobber=yes")
run_comm(comm,'Merging 4 chips')
# To inspect the final evt file (similar to fstruct/fv/fdump commands)
f = fits.open(combined_file)
f.info()
print(f[1].columns.names)

In [ ]:
# Creating image of the detector with imgev
img_file = '{}combined_img.fits'.format(prefix)
comm = (f"imgev EvtFile={combined_file} Image={img_file} CoordinateSystem=0 Projection='TAN' NAXIS1=1063 NAXIS2=1063 " +
        f"CUNIT1='deg' CUNIT2='deg' CRVAL1={RApoint} CRVAL2={DECpoint} CRPIX1=532 CRPIX2=532 " + 
        f"CDELT1=-6.207043e-4 CDELT2=6.207043e-4")
#print(comm)
run_comm(comm, "Creating Deep Field Image")

In [ ]:
# Plotting image
axs = []
image_data = fits.getdata(img_file, ext=0)
fig, _axs = plt.subplots(nrows=1, ncols=1, figsize=(16,10))
fig.suptitle("SImulation of the CDFS (5 ks)")
axs.append(_axs)

cdelt = 6.207043e-4
crpix = 532

# Add secondary axes with coordinates (optional)
ra1 = RApoint + crpix*cdelt/np.cos(radians(DECpoint))
ra2 = RApoint - crpix*cdelt/np.cos(radians(DECpoint))
dec1 = DECpoint - crpix*cdelt
dec2 = DECpoint + crpix*cdelt

sq_downleft = np.array((ra1, dec1))
sq_upright = np.array((ra2, dec2))
draw_second_axis(fig, axs, sq_downleft=sq_downleft, sq_upright=sq_upright, ln=1063, mov_ra=20, mov_dec=5, 
                 lon_lines=17, num_lines=10, mov_x_text=[0, 67], mov_y_text=[17, 20], 
                 xlabel=r'$\alpha$/deg', ylabel=r'$\delta$/deg', num_axis=0, dist_txt=8, 
                 zero_between=False, display_format='.2f')


plt.ylim(0, len(image_data))
#cmap = plt.cm.winter
cmap = plt.cm.Blues_r
im = axs[0].imshow(image_data, cmap=cmap, norm=colors.LogNorm(vmin=0.1, vmax=np.max(image_data), clip=True))
fig.colorbar(im,ax=_axs)

## 10.4.4 Dithering and Exposure Map

In [ ]:
# Generate an Attitude file suited for CDFS
att_file = "attitude_lissajous.fits"
comm = (f"attgen_dither Attitude={att_file} Amplitude=0.035 SrcRA={RApoint} SrcDec={DECpoint} Exposure=5000")
#print(comm)
run_comm(comm, "Generating Attitude File for CDFS")

In [ ]:
# Obtaining exposure map:  5 ks snapshot of the attitude at a lower time resolution
expo_map = 'expo_map.fits'
vig_file = f"{xmldir}/athena_vig_15row_20171016.fits"
comm = (f'exposure_map Vignetting={vig_file} Attitude={att_file} Exposuremap={expo_map} XMLFile="{xml0};{xml1};{xml2};{xml3}" ' +
        f'fov_diameter=70 CoordinateSystem=0 projection_type=TAN NAXIS1=1063 NAXIS2=1063 CUNIT1=deg CUNIT2=deg ' +
        f'CRVAL1={RApoint} CRVAL2={DECpoint} CRPIX1=532 CRPIX2=532 CDELT1=-6.207043e-04 CDELT2=6.207043e-04 ' + 
        f'TSTART=0 timespan=5000.000000 dt=100. chatter=3 clobber=true')
#print(comm)
run_comm(comm, 'Calculating exposure map')

In [ ]:
# plot attitude file and plot exposure map
# read coords of attitude file
f = fits.open(att_file)
ra = f[1].data['RA']
dec = f[1].data['Dec']
f.close()
fig, _axs = plt.subplots(nrows=1, ncols=2, figsize=(16*2,6*2))
axs = _axs.flatten()
axs[0].plot(ra, dec, 'r-')
axs[0].set_xlabel('RA [deg]')
axs[0].set_ylabel('Dec [deg]')
axs[0].title.set_text('Fig19 Left: Satellite pointing')


f = fits.open(expo_map)
data = f[0].data
f.close()
cmap = plt.cm.YlOrBr_r
im = axs[1].imshow(data, cmap=cmap)
fig.colorbar(im,ax=axs)

draw_second_axis(fig, axs, sq_downleft=sq_downleft, sq_upright=sq_upright, ln=1063, mov_ra=20, mov_dec=5, 
                 lon_lines=17, num_lines=10, mov_x_text=[0, 67], mov_y_text=[17, 20], 
                 xlabel=r'$\alpha$/deg', ylabel=r'$\delta$/deg', num_axis=1, dist_txt=23, 
                 zero_between=False, display_format='.2f')
axs[1].set_ylim(0, len(image_data))
axs[1].set_xlim(0, len(image_data))
axs[1].set_xticklabels([])
axs[1].set_yticklabels([])
axs[1].title.set_text('Fig19 right: Exposure map')

In [ ]:
# Run new simulation to remove the detector gaps
comm = (f'athenawfisim Attitude={att_file} Prefix=c_att_ XMLFile0={xml0} XMLFile1={xml1} XMLFile2={xml2} XMLFile3={xml3} ' + 
        f'Simput=CDFS_cat_lehmer.fits Simput2=CDFS_cat_galaxies.fits Exposure=5000 clobber=yes')
#print(comm)
run_comm(comm, 'Simulating 4 chips')

In [ ]:
# merge chips
merged_combined_nogap = 'c_att_combined_evt.fits'
comm = (f'ftmerge c_att_chip0_evt.fits,c_att_chip1_evt.fits,c_att_chip2_evt.fits,c_att_chip3_evt.fits {merged_combined_nogap} clobber=yes')
run_comm(comm, 'Merging 4 event files - no gaps')

In [ ]:
# Create image
img_file = 'c_att_combined_img.fits'
comm = (f'imgev EvtFile={merged_combined_nogap} Image={img_file} CoordinateSystem=0 Projection=TAN NAXIS1=1063 ' +
        f'NAXIS2=1063 CUNIT1=deg CUNIT2=deg CRVAL1={RApoint} CRVAL2={DECpoint} CRPIX1=532 CRPIX2=532 ' + 
        f'CDELT1=-6.207043e-04 CDELT2=6.207043e-04 history=true clobber=yes')
#print(comm)
run_comm(comm, 'Creating no gaps image')

In [ ]:
# Plotting Image no gaps
axs = []
image_data = fits.getdata(img_file, ext=0)
fig, _axs = plt.subplots(nrows=1, ncols=1, figsize=(16,10))
axs.append(_axs)
draw_second_axis(fig, axs, sq_downleft=sq_downleft, sq_upright=sq_upright, ln=1063, 
                 mov_ra=20, mov_dec=5, lon_lines=17, num_lines=10, mov_x_text=[0, 67], 
                 mov_y_text=[17, 20], xlabel=r'$\alpha$/deg', ylabel=r'$\delta$/deg', 
                 num_axis=0, dist_txt=23, zero_between=False, display_format='.2f')

plt.xlim(0, len(image_data))
plt.ylim(0, len(image_data))
cmap = plt.cm.winter
cmap = plt.cm.Blues_r
im = axs[0].imshow(image_data, cmap=cmap, norm=colors.LogNorm(vmin=np.min(image_data)+0.1, vmax=np.max(image_data), clip=True))
fig.colorbar(im,ax=_axs)